# Fairness Assessment

In [17]:
import numpy as np
import pandas as pd
import pickle
 
import datetime
dte = datetime.datetime.now()
dte = dte.strftime("%Y%m%d")

# import custom functions
import sys
sys.path.append('/ML-AI-PCOR-2021/onc_functions')

from fairness import get_fairness_assessment

print('python-' + sys.version)

python-3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


In [11]:
# load pooled results from the xgboost imputed model (not calibrated)

pred_df = pd.read_csv('./roc_results/2021_xgb_pooling_results_final_roc.csv')
pred_df = pred_df.loc[:,['averaged','died_in_90','usrds_id']].copy()
pred_df.head()

,averaged,died_in_90,usrds_id
0,0.467093,1,31089.0
1,0.473195,0,34521.0
2,0.290782,0,46751.0
3,0.225663,0,50506.0
4,0.041878,0,54985.0


In [12]:
#load fairness details
with open('../complete_fairness_data.pickle', 'rb') as f:
        dataset = pickle.load(f)
dataset.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,2969676.0,0,72.0,2.0,1.0,2.0,2.0,8
1,2969869.0,1,73.0,2.0,1.0,2.0,2.0,9
2,2970021.0,0,55.0,2.0,1.0,1.0,1.0,8
3,2970067.0,0,72.0,1.0,1.0,1.0,2.0,9
4,2970686.0,0,49.0,2.0,1.0,2.0,2.0,8


In [13]:
#combine these 2 dataframes
data = pred_df.merge(dataset, how='left', on=['usrds_id','died_in_90'])
data.head()

,averaged,died_in_90,usrds_id,inc_age,sex,dialtyp,race,hispanic,subset
0,0.467093,1,31089.0,80.0,1.0,1.0,2.0,2.0,9
1,0.473195,0,34521.0,64.0,2.0,1.0,1.0,2.0,8
2,0.290782,0,46751.0,69.0,2.0,1.0,1.0,2.0,8
3,0.225663,0,50506.0,73.0,2.0,NaN,1.0,2.0,9
4,0.041878,0,54985.0,48.0,1.0,1.0,1.0,2.0,8


In [14]:
fairness = get_fairness_assessment(
                                    data,
                                    y_proba_col_name='averaged',
                                    y_true_col_name='died_in_90')
fairness.head(39)

,Feature,Value,Count,AUC,TN,FP,FN,TP
0,agegroup,1.0,4340,0.868198,4288,6,42,4
1,agegroup,2.0,12774,0.845767,12527,35,186,26
2,agegroup,3.0,26120,0.847524,25362,177,483,98
3,agegroup,4.0,53564,0.818923,51122,627,1555,260
4,agegroup,5.0,85076,0.800299,79018,1734,3540,784
5,agegroup,6.0,86140,0.785300,74566,4050,5413,2111
6,agegroup,7.0,62193,0.765894,47085,6840,4697,3571
7,agegroup,8.0,15098,0.750794,9313,2817,1268,1700
8,sex,1.0,198347,0.830897,174200,9500,9565,5082
9,sex,2.0,146957,0.819263,129080,6786,7619,3472


In [15]:
#save (for the aucroc evaluated model)
fairness.to_csv('./roc_results/' + str(dte) + '_xgb_imputed_fairness.csv')